# Настройка окружения

In [2]:
!pip install gigachat -U -q


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [11]:
from rich import print
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

# Обращение к GigaChat

In [20]:
from gigachat import GigaChat

llm = GigaChat(model="GigaChat-2-Max", profanity_check=False, timeout=120)
resp = llm.chat("Кто тебя сделал?")
print(resp.choices[0].message.content)

Меня создала компания Sber в России в 2023 году.

In [21]:
print(resp)

ChatCompletion(
    x_headers={
        'x-request-id': '6411ff36-de8c-427b-a177-2b642511c58d',
        'x-session-id': '8ff0b5a3-93b0-4bb9-ac88-739af0490b52',
        'x-client-id': None
    },
    choices=[
        Choices(
            message=Messages(
                role='assistant',
                content='Меня создала компания Sber в России в 2023 году.',
                function_call=None,
                name=None,
                attachments=None,
                data_for_context=None,
                functions_state_id=None,
                id_=None
            ),
            index=0,
            finish_reason='stop'
        )
    ],
    created=1750249764,
    model='GigaChat-2-Max:2.0.28.02',
    thread_id=None,
    message_id=None,
    usage=Usage(prompt_tokens=1, completion_tokens=17, total_tokens=18, precached_prompt_tokens=15),
    object_='chat.completion'
)

# Чат-бот без памяти

In [24]:
while True:
    q = input("User: ")
    if q == "":
        break
    print(f"User: {q}")
    resp = llm.chat(q)
    print(f"Giga: {resp.choices[0].message.content}")

User: Привет

Giga: Привет!

User: Меня зовут Константин

Giga: Приятно познакомиться, Константин! Чем займёмся?

User: Как меня зовут?

Giga: Ты не представился(лась), поэтому я пока не знаю твоего имени. Скажи, как тебя зовут!

# Чат-бот с памятью

In [ ]:
from gigachat.models import Messages, MessagesRole, Chat

messages = []  # История сообщений

while True:
    q = input("User: ")
    if q == "":
        break
    print(f"User: {q}")
    messages.append(Messages(role=MessagesRole.USER, content=q))
    answer = llm.chat(Chat(messages=messages)).choices[0].message
    print(f"Giga: {answer.content}")
    messages.append(answer)

User: Привет!

Giga: Привет! Как настроение?

User: Меня зовут Константин

Giga: Приятно познакомиться, Константин! Чем займёмся?

User: Как меня зовут?

Giga: Тебя зовут Константин.

User: Что ты только что сказал?

Giga: Только что я подтвердил твоё имя — тебя действительно зовут Константин.

# Вызов функции моделью

## 1. Обращение к модели с функцией поиска

In [59]:
from gigachat.models import Function, FunctionParameters

search = Function(
        name="search",
        description="""Выполняет поиск в интернете.""",
        parameters=FunctionParameters(
            type="object",
            properties={"query": {"type": "string", "description": "Поисковый запрос"}},
            required=["query"],
        ),
    )

messages = [
    Messages(role=MessagesRole.USER, content="Какая самая дорогая компания в мире в 2025?")
]

resp = llm.chat(Chat(messages=messages, functions=[search])).choices[0].message
messages.append(resp)
print(resp)

Messages(
    role='assistant',
    content='',
    function_call=FunctionCall(name='search', arguments={'query': 'самая дорогая компания в мире в 2025'}),
    name=None,
    attachments=None,
    data_for_context=None,
    functions_state_id='4a5d4179-781c-4d04-b050-21494c6f0c80',
    id_=None
)

## 2. Обработка ответа модели

In [60]:
def my_search_function(query: str):
    print(f">> my_search_function called with query: {query}")
    return "Самая дорогая компания в мире в 2025 - компания XYZ с капитализацией 3 триллиона долларов."

func_result = None

if resp.function_call:
    name = resp.function_call.name
    if name == 'search':
        query = resp.function_call.arguments['query']
        func_result = my_search_function(query)


>> my_search_function called with query: самая дорогая компания в мире в 2025

## 3. Возвращаем модели результаты поиска

In [61]:
import json

messages.append(Messages(role=MessagesRole.FUNCTION,
                         content=json.dumps({"result": func_result}, ensure_ascii=False)))

In [62]:
resp = llm.chat(Chat(messages=messages, functions=[search])).choices[0].message
print(resp)

Messages(
    role='assistant',
    content='На данный момент самой дорогой компанией в мире считается **XYZ**, чья рыночная капитализация достигла
отметки в **3 триллиона долларов** (по состоянию на 2025 год). \n\nЭта информация основана на актуальных данных, 
однако стоит учитывать, что ситуация на рынке меняется ежедневно, и лидерство среди компаний также может 
варьироваться в зависимости от экономических условий и успехов отдельных корпораций.',
    function_call=None,
    name=None,
    attachments=None,
    data_for_context=None,
    functions_state_id='c4618783-9661-4203-ad75-4565fc5ff47f',
    id_=None
)

# Собираем все в полноценного агента

## 1. Настоящий поиск в интернете

In [63]:
!pip install duckduckgo_search -q -U

2289.69s - pydevd: Sending message related to process being replaced timed-out after 5 seconds



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [70]:
from langchain_community.tools.ddg_search.tool import DuckDuckGoSearchRun

def my_search_function(query: str):
    print(f">> my_search_function called with query: {query}")
    return DuckDuckGoSearchRun().run(query)

## 2. Агентный цикл

In [78]:
def run_agent(functions):
    messages = []  # История сообщений
    func_result = None
    
    while True:
        if func_result:
            print(f"Function result: {func_result[0:100]}...")
            messages.append(Messages(
                role=MessagesRole.FUNCTION, 
                content=json.dumps({"result": func_result}, ensure_ascii=False)))
            func_result = None
        else:
            q = input("User: ")
            if q == "":
                break
            print(f"User: {q}")
            messages.append(Messages(role=MessagesRole.USER, content=q))
            
        resp = llm.chat(Chat(messages=messages, functions=functions)).choices[0].message
        messages.append(resp)

        if resp.function_call:
            name = resp.function_call.name
            if name == 'search':
                query = resp.function_call.arguments['query']
                func_result = my_search_function(query)
            else:
                raise ValueError(f"Unknown function call: {name}")
        else:    
            print(f"Giga: {resp.content}")
            
run_agent(functions=[search])

User: Какая погода будет в августе во Владивастоке?

>> my_search_function called with query: погода во Владивостоке в августе 2023

Function result: Погода во Владивостоке в августе 🌡️. Долгосрочный прогноз погоды во Владивостоке, Приморский край,
Р...

Giga: Во Владивостоке в августе ожидается достаточно тёплая и преимущественно сухая погода:

- **Дневная температура**: около +25°C (иногда поднимается до +35°C).
- Количество осадков ниже среднего уровня, преобладают солнечные дни.
- Возможны отдельные кратковременные дожди и ветер.

Температура моря прогревается примерно до +24...+26°C, делая этот период идеальным для отдыха на побережье. Однако 
стоит учитывать вероятность отдельных неблагоприятных явлений вроде сильных ветров или ливней, особенно ближе к 
концу месяца.